In [1]:
import os
from RISparser import readris
import gensim
from gensim import models
import pprint
from nltk.stem import WordNetLemmatizer
from pprint import pprint

In [2]:
from gensim.parsing.preprocessing import STOPWORDS
stopwords = set(STOPWORDS)
my_stopwords = {'approach','paper','article','model','viewpoint','conclusion','discussion','method','introduction','section','framework','based','result','proposed','theory','information','process','analysis','data','problem','review','springer','verlag'}
my_stopwords = set.union(stopwords,my_stopwords)
my_stopwords = frozenset(my_stopwords)

In [3]:
data_folder = 'D:/Pesquisa/NLP_LDA_InSySPo/data/'
files = os.listdir(data_folder)

In [26]:
texts = []
dois = []
author_address = []
publication_year = []
lem = WordNetLemmatizer()

In [84]:
for ff in files:
    f = data_folder + ff
    fid = open(f,'r',encoding="utf8")
    data = readris(fid)

    for i,entry in enumerate(data):
    
        try:
            title = entry['title']
            abstract = entry['abstract']
            
            if 'doi' in entry:
                dois += [entry['doi']]
            else:
                dois += [['']]

            if 'author_address' in entry:
                country = entry['author_address'].split(',')[-1]
                if country[0] == ' ':
                    country = country[1:]
                author_address += [country]
            else:
                author_address += ['-']

            if 'publication_year' in entry:
                publication_year += [entry['publication_year']]
            else:
                publication_year += ['-']

            text = title + ' ' + abstract
            text = gensim.utils.simple_preprocess(text)
            for i,t in enumerate(text):
                text[i] = lem.lemmatize(t)

            text = [word for word in text if word not in my_stopwords]

            texts += [text]

        except:
            pass

    fid.close()


In [85]:
dictionary = gensim.corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in texts]

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=4, workers=4)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.026*"decision" + 0.014*"support" + 0.013*"management" + 0.011*"agent" + 0.010*"system" + 0.009*"process" + 0.008*"making" + 0.008*"student" + 0.008*"service" + 0.007*"study"
Topic: 1 
Words: 0.028*"game" + 0.025*"agent" + 0.016*"virtual" + 0.016*"learning" + 0.010*"behavior" + 0.008*"player" + 0.007*"training" + 0.007*"team" + 0.007*"software" + 0.006*"environment"
Topic: 2 
Words: 0.028*"rule" + 0.021*"logic" + 0.020*"reasoning" + 0.018*"theory" + 0.016*"fuzzy" + 0.015*"system" + 0.010*"representation" + 0.010*"knowledge" + 0.009*"inference" + 0.009*"model"
Topic: 3 
Words: 0.022*"human" + 0.013*"health" + 0.011*"intelligence" + 0.011*"artificial" + 0.009*"robot" + 0.007*"care" + 0.007*"study" + 0.007*"cognitive" + 0.006*"covid" + 0.006*"social"
Topic: 4 
Words: 0.055*"network" + 0.017*"detection" + 0.017*"data" + 0.014*"fault" + 0.013*"sensor" + 0.011*"neural" + 0.011*"proposed" + 0.011*"time" + 0.009*"monitoring" + 0.007*"learning"
Topic: 5 
Words: 0.033*"knowledg

In [86]:
topics = []
scores = []
for ind_doc, bc in enumerate(bow_corpus):
    ind_top, score = sorted(lda_model[bc], key=lambda tup: -1*tup[1])[0]
    topics += [ind_top]
    scores += [score]

In [87]:
ind_doc = 6
topics[ind_doc]
scores[ind_doc]

0.29752576

In [93]:
teste = 450
print('\nTitle & Abs: {}\Country: {}\nDOI:{}\nTopic: {} - Score: {}\n'.format(texts[teste], author_address[teste], dois[teste], topics[teste], scores[teste]))


Title & Abs: ['vlsi', 'implementation', 'fuzzy', 'inference', 'engine', 'expert', 'chip', 'present', 'vlsi', 'implementation', 'inference', 'mechanism', 'cope', 'uncertainty', 'perform', 'approximate', 'reasoning', 'design', 'max', 'min', 'operation', 'fuzzy', 'set', 'theory', 'effective', 'real', 'time', 'use', 'inference', 'mechanism', 'handle', 'imprecise', 'uncertain', 'knowledge', 'obtain', 'human', 'expert', 'knowledge', 'simulate', 'reasoning', 'process', 'inference', 'mechanism', 'realized', 'custom', 'cmos', 'technology', 'emphasizes', 'simplicity', 'extensibility', 'efficiency', 'timing', 'simulation', 'suggests', 'inference', 'engine', 'perform', 'approximately', 'fuzzy', 'logical', 'inference', 'second', 'potential', 'application', 'inference', 'engine', 'real', 'time', 'decision', 'making', 'area', 'command', 'control', 'adaptive', 'command', 'generation', 'robotic', 'system']\Country: United States
DOI:10.1016/0020-0255(86)90017-4
Topic: 2 - Score: 0.3471664488315582



In [95]:
print(len(topics),len(author_address),len(texts),len(dois))

248368 245262 248368 248370


In [13]:
ind_doc = 430
for index, score in sorted(lda_model[bow_corpus[ind_doc]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic ({}): {}".format(score, index, lda_model.print_topic(index, 10)))


Score: 0.5360517501831055	 
Topic (8): 0.042*"algorithm" + 0.016*"problem" + 0.014*"proposed" + 0.013*"optimization" + 0.011*"search" + 0.008*"result" + 0.008*"space" + 0.007*"local" + 0.007*"performance" + 0.006*"swarm"

Score: 0.1691380739212036	 
Topic (2): 0.051*"image" + 0.023*"recognition" + 0.018*"object" + 0.013*"human" + 0.012*"sensor" + 0.011*"visual" + 0.009*"vision" + 0.008*"video" + 0.008*"feature" + 0.008*"motion"

Score: 0.12600445747375488	 
Topic (10): 0.036*"feature" + 0.033*"classification" + 0.025*"machine" + 0.024*"learning" + 0.019*"classifier" + 0.016*"data" + 0.015*"algorithm" + 0.014*"accuracy" + 0.014*"vector" + 0.012*"performance"

Score: 0.10859081894159317	 
Topic (15): 0.029*"gene" + 0.022*"brain" + 0.013*"expression" + 0.013*"signal" + 0.009*"analysis" + 0.009*"subject" + 0.008*"study" + 0.008*"data" + 0.007*"eeg" + 0.006*"pattern"

Score: 0.04393729940056801	 
Topic (13): 0.034*"learning" + 0.034*"segmentation" + 0.014*"reinforcement" + 0.009*"channel" 

In [28]:
len(bow_corpus)

NameError: name 'bow_corpus' is not defined